##### 네이버 중고나라 크롤링
- http://cafe.naver.com/joonggonara
- 중고나라 검색 키워드를 입력받아 판매중인 상품에 대한 제목, 링크, 조회수, 등록날짜, 가격을 크롤링하여 데이터 프레임 만들기
- 모바일 페이지를 활용하세요


- get_total : 전체 데이터 갯수를 가져오는 함수
- get_items : 아이템 리스트를 가져오는 함수
- get_price : 상세 페이지에서 가격정보를 가져오는 함수
- make_datas : 아이템을 데이터 리스트로 만드는 함수 (판매 중인 데이터만)
- all_datas : 키워드를 입력받아 전체 데이터를 가져오는 함수

In [1]:
import requests
import pandas as pd 
from bs4 import BeautifulSoup

In [2]:
# 전체 데이터 개수를 가져오는 함수
def get_total(keyword):
    url = "https://m.cafe.naver.com/ArticleSearchList.nhn?search.query=" + keyword + \
    "&search.menuid=&search.searchBy=0&search.sortBy=date&search.clubid=10050146&search.option=0&search.defaultValue=1"
    
    response = requests.get(url)
    dom = BeautifulSoup(response.content, "html.parser")
    total = dom.select_one("#ct div.sort_l span").text
    
    return total

# 테스트 코드
get_total("트레이더스 에어프라이어")

'534'

In [3]:
# 아이템 리스트를 가져오는 함수
def get_items(keyword, page):
    url = "https://m.cafe.naver.com/ArticleSearchListAjax.nhn?search.query=" + keyword +\
    "&search.menuid=&search.searchBy=1&search.sortBy=date&search.clubid=10050146&search.option=0&search.defaultValue=&search.page={}".format(page)
    
    response = requests.get(url)
    dom = BeautifulSoup(response.content, "html.parser")
    
    return dom.select("ul li")

# 테스트 코드
items = get_items("트레이더스 에어프라이어", 1)    
len(items)

20

In [4]:
# 상세 페이지에서 가격정보를 가져오는 함수
def get_price(link):
    
    headers = {
        "cookie": "npic=/22q6gTqb8m5rLRtMLJb2j6syV65Rp3Lap36es8xgb658MuAMANe7igziEZmodg9CA==; ASID=731781b800000160f33adabe0000004f; NNB=ATMZITJ6EB4VU; nsr_acl=1; NFS=1; nx_ssl=2; nid_iplevel=1; NV_YA9_TNO=-99; nid_inf=421342526; NID_AUT=ZODjEayRPdowXBFwwuECy6qDJO9EJYnZJz5w4IYY3p+nYbEyHfMXyAe4njb4TQZV; NID_JKL=jU0tH2RV3JPPHoArXRGbkcJvAWk7YO1nq20oU7z+Gjs=; NID_SES=AAABsurYoSmXJ0spw3Sis4HVkNKvc9+UJ1BRpnBSa/8ha7hBQTx5Ah+HnX5KH/YTdgrflYIFotlr/NaqS2JJIdZIc0TaynaZwdkiG24qRvrYa315sag8W3B1F5T43vDgrdIwRiT0Ljh6nJKn98Fv7vnxze8YmTQQIYwBqDa/bOi3xgEBwJDDIF+/MdZWyDNcoomxmfi1KQnB4PxRKqL0CeN6t5ywNrD2XJEs9ogBFk2kR7p70PmOwUc5ba517oGsHhe2P8m+OE61DifjVlavmoqLuim232A/a7gP8mJ2Jq4HZHpZcx6ucs21s0RkczSWX8a5VZ7uhRgMA1d7Pg/7zuDrdI837I7BUtNpxDYX1z53f9nfjE5w/Cr8Qg1oDOwZ1G91s8pnsMme7GHjroCz6ttmmpl6lNY1OH29C8aa5NV2LWmvZfxpdbMrupF9JygDjKUesEUj7uh8skCrWXLiu92D5f+jPuoqHNQyzyZPUZ95jl9kEBX91dB06W3oKwQ+jeG2Ex1E3/rcbCH3J0mvoHq48ON3ZqBBhZuJfd08u3M42s3D2qWaYUP7IkvkF0i2wGb8mR743n1D7dR7824Rt/c7l/c=; BMR="
    }
    
    response = requests.get(link, headers=headers)
    dom = BeautifulSoup(response.content, "html.parser")
    
    return dom.select_one(".price em").text
    

    
# 테스트 코드
link = "https://m.cafe.naver.com/ArticleRead.nhn?clubid=10050146&menuid=451&articleid=455147331&query=%ED%8A%B8%EB%A0%88%EC%9D%B4%EB%8D%94%EC%8A%A4+%EC%97%90%EC%96%B4%ED%94%84%EB%9D%BC%EC%9D%B4%EC%96%B4"
get_price(link)

'110,000'

In [5]:
# 아이템을 데이터 리스트로 만드는 함수
def make_datas(items):
    
    datas = []
    for item in items:
        try:
            sell = item.select_one(".icon_txt").text
            if sell == "완료":
                continue
        except:
            continue
        
        title = item.select_one("h3").text
        links = "https://m.cafe.naver.com" + item.select_one("a").attrs["href"]
        view = item.select_one(".no em").text
        date = item.select_one(".time").text
        price = get_price(links)
        datas.append([title, links, view, date, price]) # 리스트로 붙이면 DataFrame 구성할 때 편함

    return datas

# 테스트 코드
datas = make_datas(items)
len(datas)

6

In [6]:
# 키워드를 입력받아 전체 데이터를 가져오는 함수
def all_datas(keyword):
    
    total = get_total(keyword)
    pages = int(total) // 20 + 1
    
    datas = []
    
    for page in range(1, pages+1):
        items = get_items(keyword, page)
        datas += make_datas(items)
        
    
    return datas
    
# 테스트 코드
datas = all_datas("트레이더스 에어프라이어")
len(datas)

101

In [7]:
# 데이터 프레임을 만드는 코드
datas = all_datas("트레이더스 에어프라이어")

columns = ["title", "link", "views", "date", "price"]
df = pd.DataFrame(datas, columns=columns)
df

,title,link,views,date,price
0,트레이더스에어프라이어 구매합니다. [AFG-1702T],https://m.cafe.naver.com/ArticleRead.nhn?clubi...,95,11:05,"100,000,000"
1,트레이더스에어프라이어 구매합니다. [AFG-1702T],https://m.cafe.naver.com/ArticleRead.nhn?clubi...,240,18.05.11.,"100,000,000"
2,(거래완료)트레이더스 에어프라이어(새제품) 팝니다,https://m.cafe.naver.com/ArticleRead.nhn?clubi...,525,18.05.10.,"12,345"
3,트레이더스 에어프라이어 구매합니다.,https://m.cafe.naver.com/ArticleRead.nhn?clubi...,210,18.05.10.,"84,800"
4,사기조심하세요 트레이더스 에어프라이어판매,https://m.cafe.naver.com/ArticleRead.nhn?clubi...,387,18.05.04.,"2,000"
5,이마트 트레이더스 에어프라이어 AFG 1702T 모델 구매원합니다.,https://m.cafe.naver.com/ArticleRead.nhn?clubi...,682,18.05.03.,"110,000"
6,트레이더스 에어프라이어 사기꾼 많으니 조심하세요~,https://m.cafe.naver.com/ArticleRead.nhn?clubi...,430,18.05.02.,"5,533,333"
7,이마트 트레이더스 에어프라이어 사기꾼 조심,https://m.cafe.naver.com/ArticleRead.nhn?clubi...,587,18.05.02.,"44,444"
8,트레이더스 에어프라이어 삽니다! 창원,https://m.cafe.naver.com/ArticleRead.nhn?clubi...,219,18.05.02.,"84,800"
9,트레이더스 에어프라이어 AFG-1702T vs 매직쉐프5L,https://m.cafe.naver.com/ArticleRead.nhn?clubi...,"1,305",18.05.02.,"96,000"
